In [ ]:
import torch
from vsa import VSA
VSA.mode = "SOFTWARE"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vec = VSA.random(size = (100, 1000), device = device)

with torch.profiler.profile(
            activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
            on_trace_ready=torch.profiler.tensorboard_trace_handler('./profiler/tests'),
            record_shapes=True,
            profile_memory=True,
            with_stack=True) as prof:
    out = VSA.multibind(vec)

print(prof.key_averages().table(row_limit=10))
out.shape

In [ ]:
vsa.bind(vectors[0], vectors[1]), vsa.bundle(vectors[0], vectors[1])

In [ ]:
vsa.multibind(vectors), vsa.multiset(vectors, False)

In [ ]:
vsa.similarity(vectors[0], vectors[1])

In [ ]:
import torch
weights = torch.tensor([1,2,3,4,-1])
torch.matmul(weights.type(torch.LongTensor), vectors.type(torch.LongTensor))
# weights * vectors.transpose(-2,-1)

In [ ]:
weights = torch.tensor([[1,2,3,4,-1], [-1,-2,-3, -4, 1]])
vectors.unsqueeze(0).repeat(2,1,1)
torch.matmul(weights.type(torch.LongTensor), vectors.type(torch.LongTensor))

In [ ]:
from main import algo1, algo2, algo3
from vsa.vsa import VSA
from vsa.resonator import Resonator
import torch
import shutil

shutil.rmtree("tests/testing/")

VSA_MODE = "HARDWARE"
NUM_VEC_SUPERPOSED = 2
ALGO = "ALGO3" # ALGO1, ALGO2, ALGO3
TRIALS = 20    # for ALGO2
DIM = 2000
FACTORS = 5
CODEVECTORS = 10
CODEVECTORS : tuple = (3,3,3,10,2)
NOISE_LEVEL = 0.0  # compositional vector noise
ITERATIONS = 5000    # max number of iterations for factorization
NORMALIZE = True   # for SOFTWARE mode. Normalize the initial estimate and the input vector (when the input is a bundled vector)
ACTIVATION = 'EXPO'  # 'NONE', 'ABS', 'NONNEG'
RESONATOR_TYPE = "SEQUENTIAL" # "CONCURRENT", "SEQUENTIAL"
ARGMAX_ABS = True
REORDER_CODEBOOKS = False # True False
if VSA_MODE == 'HARDWARE':
    NORMALIZE = None
BATCH_SIZE = 1

vsa = VSA(
    root="tests/testing/",
    mode=VSA_MODE,
    dim=DIM,
    num_factors=FACTORS,
    num_codevectors=CODEVECTORS,
    seed = 0
)

rn = Resonator(vsa, type=RESONATOR_TYPE, activation=ACTIVATION, iterations=ITERATIONS, argmax_abs=ARGMAX_ABS)
ds = VSADataset("tests/testing/", 1, vsa, algo=ALGO, num_vectors_superposed=NUM_VEC_SUPERPOSED, noise=NOISE_LEVEL)
# Test sample
label = [(0, 2, 1, 1), (1, 2, 1, 5)]
# label = [(0, 0, 2, 3), (2, 0, 0, 3)]
vector = ds.lookup_algo3(label)

labels = [label]
vectors = vector.repeat(BATCH_SIZE, 1)

codebooks = None
orig_indices = None

if REORDER_CODEBOOKS:
    codebooks, orig_indices = rn.reorder_codebooks(codebooks)

init_estimates = rn.get_init_estimates(codebooks, BATCH_SIZE)
if NORMALIZE:
    init_estimates = vsa.normalize(init_estimates)

if (NUM_VEC_SUPERPOSED == 1):
    outcome, iter = rn(vectors, init_estimates, codebooks, orig_indices)
    # Make them the same format as multi-vector for easier parsing
    outcomes = [[outcome[x]] for x in range(BATCH_SIZE)]
    convergences = [1 if iter == ITERATIONS-1 else 0] * BATCH_SIZE
    iters = [iter] * BATCH_SIZE
else:
    if ALGO == "ALGO1":
        outcomes, convergences, iters = algo1(vsa, rn, vectors, init_estimates, codebooks, orig_indices, NORMALIZE)
    elif ALGO == "ALGO2":
        outcomes, convergences, iters = algo2(vsa, rn, vectors, DIM, FACTORS, codebooks, orig_indices, NORMALIZE)
    elif ALGO == "ALGO3":
        outcomes, convergences, iters = algo3(vsa, rn, vectors, NORMALIZE)


In [ ]:
print(labels)
print(outcomes)
print(iters)

In [ ]:
from vsa.vsa_tensor import VSATensor
from vsa import vsa
import torch

VSATensor.set_mode("SOFTWARE")
x = VSATensor.random(1, 100)
print(x)
print(x.quantized)
y = VSATensor.random(2, 100)
# z = vsa.bind(x, y)
# print(z)
# print(z.quantized)
# z = vsa.multibind(y)
# print(z)
# print(z.quantized)

b = vsa.bundle(x, y)
print(b)
print(b.quantized)
c = vsa.bundle(x, y, True)
print(c)
print(c.quantized)
print(vsa.dot_similarity(x, c))
print(vsa.dot_similarity(c, c))
print(vsa.dot_similarity(b, b))



In [ ]:
# Test CA90 distributive property over bind and permute
import torch
from vsa import VSA
VSA.mode = "HARDWARE"
vecs_f1 = VSA.random(size = (2, 64))
vecs_f2 = VSA.ca90(vecs_f1)
vecs_f3 = VSA.ca90(vecs_f2)
bound_f1 = VSA.multibind(vecs_f1)
bound_f2 = VSA.multibind(vecs_f2)
bound_f3 = VSA.multibind(vecs_f3)
permute_f1 = VSA.permute(vecs_f1)
permute_f2 = VSA.permute(vecs_f2)
permute_f3 = VSA.permute(vecs_f3)

bound_f2_ca = VSA.ca90(bound_f1)
bound_f3_ca = VSA.ca90(bound_f2_ca)

permute_f2_ca = VSA.ca90(permute_f1)
permute_f3_ca = VSA.ca90(permute_f2_ca)

print((bound_f2 == bound_f2_ca).all())
print((bound_f3 == bound_f3_ca).all())
print((permute_f2 == permute_f2_ca).all())
print((permute_f3 == permute_f3_ca).all())

In [ ]:
## Test how CA90 reserves similarity for locality-preserving encoding.
## What are the similaries between the full hypervectors if we apply a similarity step function to the seeds only.
## Kleyko's paper shows the power of 2 steps are displaying the best similarity to its original seed (about twice of the original distance)
## This will require a much more complicated CA90 update scheme.
import torch
from vsa import VSA
VSA.mode = "HARDWARE"
FOLD_DIM = 100
DIM = 1000
NUM_FOLDS = DIM // FOLD_DIM
NUM_STEPS = 11
STEP = FOLD_DIM // (NUM_STEPS - 1)
vecs_f0 = VSA.random(size = (NUM_STEPS, FOLD_DIM))
# Set up the vectors such that consecutive items differ by STEP in similarity
for i in range(1, NUM_STEPS):
    vecs_f0[i] = vecs_f0[i-1]
    vecs_f0[i][STEP*(i-1):STEP*i] = VSA.inverse(vecs_f0[i][STEP*(i-1):STEP*i])
    print(f"Similarity between fold 0 of vec0 and vec{i} =", VSA.hamming_similarity(vecs_f0[0], vecs_f0[i]))
# print(f"Similarity between fold 0 of vec0 and vec{NUM_STEPS-1} =", VSA.hamming_similarity(vecs_f0[0], vecs_f0[NUM_STEPS-1]) / FOLD_DIM)
print()

# CA90 updates
def gen_full_vector(fold):
    """
    Generate the rest of the vector through CA90
    """
    assert(fold.size(-1) == FOLD_DIM)

    vector = fold.clone()
    for i in range(DIM // FOLD_DIM - 1):
        fold = VSA.ca90(fold)
        vector = torch.cat((vector, fold), dim=-1)
    return vector

vecs_full = gen_full_vector(vecs_f0)

MID_FOLD = 8
for i in range(1, NUM_STEPS):
    print(f"Similarity between fold {MID_FOLD} of vec0 and vec{i} =", VSA.hamming_similarity(vecs_full[0][MID_FOLD*FOLD_DIM:(MID_FOLD+1)*FOLD_DIM], vecs_full[i][MID_FOLD*FOLD_DIM:(MID_FOLD+1)*FOLD_DIM]))
print()

for i in range(1, NUM_STEPS):
    print(f"Full similarity bewteen vec0 and vec{i} =", VSA.hamming_similarity(vecs_full[0], vecs_full[i]))
# print(f"Full similarity bewteen vec0 and vec{NUM_STEPS-1} =", VSA.hamming_similarity(vecs_full[0], vecs_full[NUM_STEPS-1]))


In [1]:
# TODO: There's a obscure bug here. The unnoisy version of the input sometimes fails to produce the correct answer.
# The same setup has been tested in the main program and the accuracy is 100% all the time.
# Not sure whether it's a typo or a bug hidden in the code

import torch
from vsa import VSA
from dataset import VSADataset
import random
from vsa.resonator import Resonator
# Generate codebooks of 1st fold only
root = "tests/folding"
N_SUPERPOSED = 1
NUM_FACTORS = 4
FOLD_DIM = 128
DIM = 2048
NUM_FOLDS = DIM // FOLD_DIM
NOISE = 0
vsa = VSA(root=root, mode="HARDWARE", dim=DIM, fold_dim = FOLD_DIM, ehd_bits=9, num_factors=NUM_FACTORS, num_codevectors=8, seed=0)
# ds = VSADataset("test/folding", 10, vsa, algo="ALGO1", num_vectors_superposed=NUM_VEC_SUPERPOSED, quantize=q, noise=n, device=device)

labels = [tuple([random.randint(0, len(vsa.codebooks[i])-1) for i in range(NUM_FACTORS)]) for j in range(N_SUPERPOSED)]
print(labels)

rn = Resonator(vsa=vsa, mode="HARDWARE", type="SEQUENTIAL", activation="THRESHOLD", iterations=5000, 
               argmax_abs=True, act_val=8, stoch="SIMILARITY", 
               randomness=0.04, early_converge=0.8, seed=0)

_query = vsa.get_vector(labels, quantize=True)
print("Original Full Vector")
print(rn(_query, rn.get_init_estimates()))

query = vsa.apply_noise(_query, NOISE)
print(f"Similarity after noise:", vsa.hamming_similarity(query, _query) / DIM)
print("Noisy Full Vector")
print(rn(query, rn.get_init_estimates()))

cb_fold = [0] * NUM_FOLDS
query_fold = [0] * NUM_FOLDS
for i in range(NUM_FOLDS):
    cb_fold[i] = vsa.codebooks[:,:,i*FOLD_DIM:(i+1)*FOLD_DIM]
    # query_fold[i] = vsa.get_vector(labels, codebooks = cb_fold[i], quantize = True)
    query_fold[i] = query[i*FOLD_DIM:(i+1)*FOLD_DIM] # Noisy version
# CA90 expands from noisy first fold
query_ca = vsa._gen_full_vector(query_fold[0])

init_est_full = rn.get_init_estimates()
print("Similarity CA90 Expanded Full Vector:", vsa.hamming_similarity(_query, query_ca) / DIM)
print(rn(query_ca, init_est_full))

for i in range(NUM_FOLDS):
    print(f"Fold {i} Similarity ca against noise-free query", vsa.hamming_similarity(_query[i*FOLD_DIM:(i+1)*FOLD_DIM], query_ca[i*FOLD_DIM:(i+1)*FOLD_DIM]) / FOLD_DIM)
    print(f"Fold {i} Similarity ca against noisy query", vsa.hamming_similarity(query_fold[i], query_ca[i*FOLD_DIM:(i+1)*FOLD_DIM]) / FOLD_DIM)
    # print(rn(query_fold[i], rn.get_init_estimates(cb_fold[i]), cb_fold[i]))



Generating codebooks...Done. Saved to tests/folding/codebooks.pt
[(7, 4, 2, 7)]
Original Full Vector
((7, 4, 2, 7), 7, 'EARLY')
Similarity after noise: tensor(1.)
Noisy Full Vector
((7, 4, 2, 7), 13, 'EARLY')
Similarity CA90 Expanded Full Vector: tensor(1.)
((7, 4, 2, 7), 25, 'EARLY')
Fold 0 Similarity ca against noise-free query tensor(1.)
Fold 0 Similarity ca against noisy query tensor(1.)
Fold 1 Similarity ca against noise-free query tensor(1.)
Fold 1 Similarity ca against noisy query tensor(1.)
Fold 2 Similarity ca against noise-free query tensor(1.)
Fold 2 Similarity ca against noisy query tensor(1.)
Fold 3 Similarity ca against noise-free query tensor(1.)
Fold 3 Similarity ca against noisy query tensor(1.)
Fold 4 Similarity ca against noise-free query tensor(1.)
Fold 4 Similarity ca against noisy query tensor(1.)
Fold 5 Similarity ca against noise-free query tensor(1.)
Fold 5 Similarity ca against noisy query tensor(1.)
Fold 6 Similarity ca against noise-free query tensor(1.)
Fol